In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import itertools
import numpy as np

In [5]:
#scraper

tours = ['tour-de-france', 'giro-d-italia', 'vuelta-a-espana']
#years = [2020, 2019, 2018, 2017, 2016, 2015, 2014]
years = [2022]#, 2021, 2013, 2012, 2011, 2010]
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]

In [6]:
def scrape_participants(tour, year):
    # define url for startlist
    
    url = f'https://www.procyclingstats.com/race/{tour}/{year}/stage-21/startlist'
    print(url)
    
    #scrape page
    response = requests.get(url).content
    soup = BeautifulSoup(response)
    
    #get all_teams
    all_teams = soup.find_all('li', class_='team')
    
    #loop over teams
    master_ls = []
    for t in all_teams:
        team = t.a.text
        riders = t.find_all('a', class_='blue')
        for r in riders:
            dict = {}
            rider = r.span.text
            href = r['href']
            dict['rider'] = href.split('/')[-1]
            dict['team'] = team
            dict['href'] = href
            dict['tour'] = tour
            dict['year'] = int(year)
            master_ls.append(dict)
            
    return master_ls

In [7]:
participants_ls = []

for y, t in list(itertools.product(years, tours)):
    participants_ls.append(scrape_participants(t, y))

https://www.procyclingstats.com/race/tour-de-france/2000/stage-21/startlist
https://www.procyclingstats.com/race/giro-d-italia/2000/stage-21/startlist
https://www.procyclingstats.com/race/vuelta-a-espana/2000/stage-21/startlist
https://www.procyclingstats.com/race/tour-de-france/2001/stage-21/startlist
https://www.procyclingstats.com/race/giro-d-italia/2001/stage-21/startlist
https://www.procyclingstats.com/race/vuelta-a-espana/2001/stage-21/startlist
https://www.procyclingstats.com/race/tour-de-france/2002/stage-21/startlist
https://www.procyclingstats.com/race/giro-d-italia/2002/stage-21/startlist
https://www.procyclingstats.com/race/vuelta-a-espana/2002/stage-21/startlist
https://www.procyclingstats.com/race/tour-de-france/2003/stage-21/startlist
https://www.procyclingstats.com/race/giro-d-italia/2003/stage-21/startlist
https://www.procyclingstats.com/race/vuelta-a-espana/2003/stage-21/startlist
https://www.procyclingstats.com/race/tour-de-france/2004/stage-21/startlist
https://www.

In [8]:
pd.DataFrame(list(itertools.chain(*participants_ls))).to_csv('new_data/participants_10s.csv')

In [9]:
participants_df = pd.DataFrame(list(itertools.chain(*participants_ls)))

In [10]:
from matplotlib.pyplot import text


def scrape_performance(rider, endpoint, year):
    
    #set up
    base_url = 'https://www.procyclingstats.com/'
    url = base_url+endpoint+'/'+str(year)
    
    response = requests.get(url).content
    soup = BeautifulSoup(response)
    
    result_ls = []
    
    #get stage_race results
    stage_races = soup.find_all('tr', {'data-main': '0'})
    
    for o in stage_races:
        dict = {}
        o = o.find_all('td')
        dict['name'] = rider
        dict['year'] = str(year)
        dict['type'] = 'etappe'
        dict['date'] = o[0].text
        if len(dict['date']) == 0:
            dict['type'] = 'gc'
        dict['result'] = o[1].text
        dict['gc'] = o[2].text
        try:
            dict['icon'] = o[3].find('span', class_='icon')['class'][-1]
        except TypeError:
            dict['icon'] = 'stage'
        dict['race_ref'] = o[4].a['href']
        dict['race_name'] = dict['race_ref'].split('/')[1]
        dict['race_detail'] = o[4].a.text
        try:
            dict['race_rank'] = o[4].a.span.text
        except AttributeError:
            dict['race_rank'] = o[4].a.span
        dict['distance'] = o[5].text
        result_ls.append(dict)
    
    #get one day race results
    one_day_races = soup.find_all('tr', {'data-main': '1'})
    
    for o in one_day_races:
        dict = {}
        o = o.find_all('td')
        dict['name'] = rider
        dict['year'] = str(year)
        dict['type'] = 'one_day'
        dict['date'] = o[0].text
        dict['result'] = o[1].text
        dict['gc'] = o[2].text
        try:
            dict['icon'] = o[3].find('span', class_='icon')['class'][-1]
        except TypeError:
            dict['icon'] = 'stage'
        dict['race_ref'] = o[4].a['href']
        dict['race_name'] = dict['race_ref'].split('/')[1]
        dict['race_detail'] = o[4].a.text
        try:
            dict['race_rank'] = o[4].a.span.text
        except AttributeError:
            dict['race_rank'] = o[4].a.span
        dict['distance'] = o[5].text
        result_ls.append(dict)
    
    return result_ls

In [11]:
performance_ls = []

for index, row in participants_df.iterrows():
    performance_ls.append(scrape_performance(row['rider'], row['href'], row['year']))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [12]:
performance_df = pd.DataFrame(list(itertools.chain(*performance_ls)))

In [20]:
performance_df.to_csv('new_data/raw_performance_10s.csv')

In [21]:
stage_s = list(np.arange(2,32,2))+list(np.arange(32,48,4))+[50]
stage_s_i = list(np.arange(1,21,1))
stage_s_dict = dict(zip(stage_s_i, stage_s[::-1]))

def clean_df(ls):
    df = pd.DataFrame(ls)
    
    index_drop = df[df['result']==''].index

    dropped_df = df.drop(index_drop)

    index_drop = dropped_df[dropped_df['type']=='gc'].index

    dropped_df = dropped_df.drop(index_drop)
    
    dropped_df['date'] = pd.to_datetime(dropped_df['date'] + '.' + dropped_df['year'], infer_datetime_format=True)
    
    dropped_df['result'] =  dropped_df['result'].replace('DNF', 0).replace('DNS', 0).replace('OTL', 0).replace('DSQ', 0).replace('DF', 0).astype('int')
    
    dropped_df['points'] = dropped_df['result'].map(stage_s_dict).fillna('0').astype('int')
    
    #depreciated -> for gc 
    #stages_df = df.loc[index_drop][['race_name', 'race_rank']]#.to_dict(orient='records')
    #stages_df = stages_df.set_index('race_name').to_dict()['race_rank']
    
    return dropped_df


In [22]:
performance_clean = clean_df(list(itertools.chain(*performance_ls)))
performance_clean.to_csv('new_data/performance_clean_10s.csv')

In [23]:
performance_clean.race_ref.unique()

array(['race/tour-de-france/2000/stage-21',
       'race/tour-de-france/2000/stage-20',
       'race/tour-de-france/2000/stage-19', ...,
       'race/vuelta-al-tachira/2009/stage-2',
       'race/vuelta-al-tachira/2009/stage-1',
       'race/nc-sweden/2009/result'], dtype=object)

In [29]:

def get_profile(list):
    extra_info_ls = []
    i=0
    
    for ref in list:
        print(i/len(performance_clean.race_ref.unique()))
        #create url
        base_url = 'https://www.procyclingstats.com/'
        url = base_url + ref
        response = requests.get(url).content
        soup = BeautifulSoup(response)
        
        print(url)
        
        #get al info
        dict = {}
        try:
            stage = soup.find('ul', class_='infolist').find_all('li')
                
            dict['href'] = ref
            #get speed
            try:
                dict[stage[2].find_all('div')[0].text] = float(stage[2].find_all('div')[1].text.strip(' km/h'))
            except ValueError:
                dict[stage[2].find_all('div')[0].text] = np.nan
            #get distance
            try:
                dict[stage[4].find_all('div')[0].text.strip()] = float(stage[4].find_all('div')[1].text.strip(' km'))
            except ValueError:
                dict[stage[4].find_all('div')[0].text.strip()] = np.nan
            #get parcours type
            try:
                dict[stage[6].find_all('div')[0].text.strip()] = stage[6].find_all('div')[1].span['class'][-1]
            except (ValueError, TypeError):
                dict[stage[6].find_all('div')[0].text.strip()] = np.nan
            #get profile score
            try:
                dict[stage[7].find_all('div')[0].text.strip()] = int(stage[7].find_all('div')[1].text)
            except ValueError:
                dict['ProfileScore:'] = np.nan
            #get vert meters
            try:
                dict[stage[8].find_all('div')[0].text.strip()] = int(stage[8].find_all('div')[1].text)
            except (ValueError, IndexError):
                dict['Vert. meters:'] = np.nan
            #get startlist
            try:
                dict[stage[12].find_all('div')[0].text.strip()] = int(stage[12].find_all('div')[1].text)
            except (ValueError, IndexError):
                dict['Startlist quality score:'] = np.nan
            #get won how
            try:
                dict[stage[13].find_all('div')[0].text]= stage[13].find_all('div')[1].text
            except (ValueError, IndexError):
                dict['Won how:'] = np.nan
            
            extra_info_ls.append(dict)
            
            i += 1
        
        except AttributeError:
            print(ref)
            
    return extra_info_ls

In [30]:
extra_info_ls = get_profile(performance_clean.race_ref.unique())

0.0
https://www.procyclingstats.com/race/tour-de-france/2000/stage-21
0.00015862944162436547
https://www.procyclingstats.com/race/tour-de-france/2000/stage-20
0.00031725888324873094
https://www.procyclingstats.com/race/tour-de-france/2000/stage-19
0.00047588832487309646
https://www.procyclingstats.com/race/tour-de-france/2000/stage-18
0.0006345177664974619
https://www.procyclingstats.com/race/tour-de-france/2000/stage-17
0.0007931472081218274
https://www.procyclingstats.com/race/tour-de-france/2000/stage-16
0.0009517766497461929
https://www.procyclingstats.com/race/tour-de-france/2000/stage-15
0.0011104060913705585
https://www.procyclingstats.com/race/tour-de-france/2000/stage-14
0.0012690355329949238
https://www.procyclingstats.com/race/tour-de-france/2000/stage-13
0.0014276649746192893
https://www.procyclingstats.com/race/tour-de-france/2000/stage-12
0.0015862944162436548
https://www.procyclingstats.com/race/tour-de-france/2000/stage-11
0.0017449238578680203
https://www.procyclingsta

In [31]:
stages_df = pd.DataFrame(extra_info_ls).rename(columns={'href':'race_ref'})
stages_df.to_csv('new_data/stages_10s.csv')

In [32]:
performance_df.drop_duplicates(inplace=True)

In [33]:
stages_df

,race_ref,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:
0,race/tour-de-france/2000/stage-21,42.991,138.0,p1,NaN,NaN,1566,Sprint of large group
1,race/tour-de-france/2000/stage-20,40.805,254.5,p1,NaN,NaN,1566,Sprint of large group
2,race/tour-de-france/2000/stage-19,53.986,58.5,p1,NaN,NaN,1566,Time Trial
3,race/tour-de-france/2000/stage-18,40.163,246.5,p1,NaN,NaN,1566,Sprint a deux
4,race/tour-de-france/2000/stage-17,45.392,155.0,p2,NaN,NaN,1566,Sprint a deux
...,...,...,...,...,...,...,...,...
6299,race/vuelta-al-tachira/2009/stage-4,41.108,120.0,p0,NaN,NaN,16,? - let us know!
6300,race/vuelta-al-tachira/2009/stage-3,44.690,78.0,p0,NaN,NaN,16,? - let us know!
6301,race/vuelta-al-tachira/2009/stage-2,42.446,140.3,p0,NaN,NaN,16,? - let us know!
6302,race/vuelta-al-tachira/2009/stage-1,45.108,110.0,p0,NaN,NaN,16,? - let us know!


In [34]:
merged = performance_clean.merge(stages_df, on='race_ref')
merged['points'] = merged['points'].astype('float')
merged['adjusted_points'] = merged['points'] * merged['ProfileScore:']  * merged['Startlist quality score:']
merged.head()

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,distance,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points
0,lance-armstrong,2000,etappe,2000-07-23,75,1,stage,race/tour-de-france/2000/stage-21,tour-de-france,Stage 21 - Paris › Paris,...,138,0.0,42.991,138.0,p1,NaN,NaN,1566,Sprint of large group,NaN
1,frankie-andreu,2000,etappe,2000-07-23,44,,stage,race/tour-de-france/2000/stage-21,tour-de-france,Stage 21 - Paris › Paris,...,138,0.0,42.991,138.0,p1,NaN,NaN,1566,Sprint of large group,NaN
2,viatcheslav-ekimov,2000,etappe,2000-07-23,16,,stage,race/tour-de-france/2000/stage-21,tour-de-france,Stage 21 - Paris › Paris,...,138,10.0,42.991,138.0,p1,NaN,NaN,1566,Sprint of large group,NaN
3,tyler-hamilton,2000,etappe,2000-07-23,76,,stage,race/tour-de-france/2000/stage-21,tour-de-france,Stage 21 - Paris › Paris,...,138,0.0,42.991,138.0,p1,NaN,NaN,1566,Sprint of large group,NaN
4,george-hincapie,2000,etappe,2000-07-23,26,,stage,race/tour-de-france/2000/stage-21,tour-de-france,Stage 21 - Paris › Paris,...,138,0.0,42.991,138.0,p1,NaN,NaN,1566,Sprint of large group,NaN


In [42]:
merged['ProfileScore:'].isna()

5636      False
5639      False
5643      False
5644      False
5646      False
          ...  
231876    False
231888    False
231892    False
231894    False
231895    False
Name: ProfileScore:, Length: 9596, dtype: bool

In [35]:
merged.to_csv('new_data/merged_clean_10s.csv')

In [36]:
merged['adjusted_points'] = merged['points'] * merged['ProfileScore:']  * merged['Startlist quality score:']
merged.dropna(subset='ProfileScore:',inplace=True)
merged.drop_duplicates(inplace=True)
merged = merged[merged['adjusted_points'] != 0]
merged.sort_values(by='adjusted_points').tail(50)

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,distance,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points
132874,carlos-sastre,2006,etappe,2006-07-19,2,2,stage,race/tour-de-france/2006/stage-16,tour-de-france,Stage 16 - Le Bourg d'Oisans › La Toussuire - ...,...,182,44.0,32.490,182.0,p5,393.0,5456.0,1426,70 km solo,24658392.0
62336,jose-vicente-garcia-acosta,2002,etappe,2002-07-23,6,113,stage,race/tour-de-france/2002/stage-15,tour-de-france,Stage 15 - Vaison-le-Romaine › Les deux Alpes,...,226.5,30.0,38.253,226.5,p5,522.0,7679.0,1575,9 km solo,24664500.0
62514,roberto-heras,2002,etappe,2002-07-18,3,14,stage,race/tour-de-france/2002/stage-11,tour-de-france,Stage 11 - Pau › La Mongie,...,158,40.0,36.190,158.0,p5,393.0,5312.0,1575,0.4 km solo,24759000.0
32363,igor-gonzalez-de-galdeano,2001,etappe,2001-07-17,8,8,stage,race/tour-de-france/2001/stage-10,tour-de-france,Stage 10 - Aix-Les-Bains › L'Alpe d'Huez,...,209,26.0,32.675,209.0,p5,630.0,8559.0,1546,7 km solo,25323480.0
92622,floyd-landis,2004,etappe,2004-07-22,5,27,stage,race/tour-de-france/2004/stage-17,tour-de-france,Stage 17 - Bourg d'Oisans › Le Grand Bornand,...,204.5,32.0,32.996,204.5,p4,505.0,8048.0,1568,Sprint of small group,25338880.0
32164,joseba-beloki,2001,etappe,2001-07-22,6,4,stage,race/tour-de-france/2001/stage-14,tour-de-france,Stage 14 - Tarbes › Luz Ardiden,...,141.5,30.0,32.098,141.5,p5,547.0,5174.0,1546,7 km solo,25369860.0
174420,samuel-sanchez,2008,etappe,2008-07-23,2,8,stage,race/tour-de-france/2008/stage-17,tour-de-france,Stage 17 - Embrun › l'Alpe d'Huez,...,210.5,44.0,34.320,210.5,p5,389.0,5405.0,1491,12.7 km solo,25519956.0
176077,leonardo-piepoli,2008,etappe,2008-07-14,1,24,stage,race/tour-de-france/2008/stage-10,tour-de-france,Stage 10 - Pau › Hautacam,...,156,50.0,36.076,156.0,p5,346.0,3794.0,1491,Sprint a deux,25794300.0
176073,juan-jose-cobo,2008,etappe,2008-07-14,1,8,stage,race/tour-de-france/2008/stage-10,tour-de-france,Stage 10 - Pau › Hautacam,...,156,50.0,36.076,156.0,p5,346.0,3794.0,1491,Sprint a deux,25794300.0
103843,georg-totschnig,2005,etappe,2005-07-16,1,14,stage,race/tour-de-france/2005/stage-14,tour-de-france,Stage 14 - Agde › Ax-3 Domaines,...,174,50.0,30.374,174.0,p5,334.0,4188.0,1549,0 km solo,25868300.0


In [38]:
merged.groupby(['race_ref']).mean().sort_values(by='adjusted_points').tail(50)

,result,points,Avg. speed winner:,Distance:,ProfileScore:,Vert. meters:,Startlist quality score:,Race category:,Points scale:,Arrival:,Won how:,adjusted_points
race_ref,,,,,,,,,,,,
race/la-route-d-occitanie/2022/stage-3,7.000000,31.333333,34.989,188.70,268.0,4503.0,298.0,NaN,NaN,NaN,NaN,2.502405e+06
race/faun-ardeche-classic/2022/result,8.600000,26.400000,37.944,168.50,216.0,3161.0,453.0,NaN,NaN,NaN,NaN,2.583187e+06
race/itzulia-basque-country/2022/stage-4,10.375000,22.750000,43.605,185.60,154.0,2703.0,742.0,NaN,NaN,NaN,NaN,2.599597e+06
race/tour-de-france/2022/stage-13,10.500000,22.100000,45.667,192.60,78.0,2109.0,1550.0,NaN,NaN,NaN,NaN,2.671890e+06
race/vuelta-a-la-comunidad-valenciana/2022/stage-1,7.714286,27.714286,38.989,166.70,162.0,3164.0,610.0,NaN,NaN,NaN,NaN,2.738726e+06
race/itzulia-basque-country/2022/stage-5,10.769231,21.076923,39.765,163.76,184.0,3485.0,742.0,NaN,NaN,NaN,NaN,2.877590e+06
race/tour-de-france/2022/stage-6,10.500000,22.100000,49.376,219.90,85.0,2477.0,1550.0,NaN,NaN,NaN,NaN,2.911675e+06
race/dauphine/2022/stage-3,8.571429,26.428571,40.620,169.00,140.0,2707.0,788.0,NaN,NaN,NaN,NaN,2.915600e+06
race/tour-de-france/2022/stage-8,10.500000,22.100000,44.164,186.30,87.0,2556.0,1550.0,NaN,NaN,NaN,NaN,2.980185e+06


In [133]:
sum(merged['ProfileScore:'].isna())

75274

In [282]:
diction = {}

stage = soup.find('ul', class_='infolist').find_all('li')
stage

[<li><div>Date:</div> <div>18 July 2021</div></li>,
 <li><div>Start time:</div> <div>16:30 </div></li>,
 <li><div>Avg. speed winner:</div> <div>40.748 km/h</div></li>,
 <li><div>Race category:</div> <div>ME - Men Elite</div></li>,
 <li><div>Distance: </div> <div>108.4 km</div></li>,
 <li><div>Points scale:</div> <div><a href="info.php?s=point-scales&amp;season=2021&amp;category=1&amp;scale=7">GT.A.Stage</a></div></li>,
 <li><div>Parcours type: </div> <div><span class="icon profile p1"></span></div></li>,
 <li><div>ProfileScore: </div> <div>14</div></li>,
 <li><div>Vert. meters:</div> <div>697</div></li>,
 <li><div>Departure:</div> <div><a href="location/chatou">Chatou</a></div></li>,
 <li><div>Arrival:</div> <div><a href="location/paris">Paris Champs-Élysées</a></div></li>,
 <li><div>Race ranking:</div> <div>1</div></li>,
 <li><div>Startlist quality score:</div> <div><a href="race/tour-de-france/2021/stage-21/startlist/lineup-quality">1646</a></div></li>,
 <li><div>Won how: </div> <div

In [292]:
diction['href'] = endpoint
#get speed
diction[stage[2].find_all('div')[0].text] = float(stage[2].find_all('div')[1].text.strip(' km/h'))
#get distance
diction[stage[4].find_all('div')[0].text.strip()] = float(stage[4].find_all('div')[1].text.strip(' km'))
#get parcours type
diction[stage[6].find_all('div')[0].text.strip()] = stage[6].find_all('div')[1].span['class'][-1]
#get profile score
diction[stage[7].find_all('div')[0].text.strip()] = int(stage[7].find_all('div')[1].text)
#get vert meters
diction[stage[8].find_all('div')[0].text.strip()] = int(stage[8].find_all('div')[1].text)
#get vert meters
diction[stage[12].find_all('div')[0].text.strip()] = int(stage[12].find_all('div')[1].text)
#get won how
diction[stage[13].find_all('div')[0].text]= stage[13].find_all('div')[1].text
diction

{'href': 'race/tour-de-france/2021/stage-21',
 'Avg. speed winner:': 40.748,
 'Distance: ': 108.4,
 'Parcours type:': 'p1',
 'ProfileScore:': 14,
 'Distance:': 108.4,
 'Vert. meters:': 697,
 'Startlist quality score:': 1646,
 'Won how: ': 'Sprint of large group'}

In [294]:
pd.DataFrame(dict)

AttributeError: 'dict' object has no attribute 'to_records'

In [259]:
stage[2].find_all('div')[0].text, stage[2].find_all('div')[1].text

('Avg. speed winner:', '40.748 km/h')

In [232]:
stages_df.set_index('race_name').to_dict()['race_rank']

{'tour-de-france': '(2.UWT)',
 'tour-of-slovenia': '(2.Pro)',
 'itzulia-basque-country': '(2.UWT)',
 'tirreno-adriatico': '(2.UWT)',
 'uae-tour': '(2.UWT)'}

In [187]:
tadej_df.drop(index_drop)

,year,type,date,result,gc,icon,race_ref,race_name,race_detail,race_rank,distance
0,2021,gc,,1,,st6,race/tour-de-france/2021/stage-21-youth,tour-de-france,Youth classification,None,
1,2021,gc,,1,,st7,race/tour-de-france/2021/stage-21-kom,tour-de-france,Mountains classification,None,
2,2021,gc,,8,,st5,race/tour-de-france/2021/stage-21-points,tour-de-france,Points classification,None,
3,2021,gc,,1,,st4,race/tour-de-france/2021/gc,tour-de-france,General classification,None,
4,2021,etappe,18.07,72,,stage,race/tour-de-france/2021/stage-21,tour-de-france,Stage 21 - Chatou › Paris Champs-Élysées,None,108.4
...,...,...,...,...,...,...,...,...,...,...,...
75,2021,one_day,20.06,5,,stage,race/nc-slovenia/2021/result,nc-slovenia,National Championships Slovenia - Road Race (NC),[(NC)],172
76,2021,one_day,17.06,3,,chrono,race/nc-slovenia-itt/2021/result,nc-slovenia-itt,National Championships Slovenia - ITT (NC),[(NC)],31.5
78,2021,one_day,25.04,1,,stage,race/liege-bastogne-liege/2021/result,liege-bastogne-liege,Liège-Bastogne-Liège (1.UWT),[(1.UWT)],259.1
79,2021,one_day,21.04,DNS,,stage,race/la-fleche-wallone/2021/result,la-fleche-wallone,La Flèche Wallonne (1.UWT),[(1.UWT)],193.6


In [164]:
tadej[len(tadej_df['date']) > 5]

{'year': '2021',
 'type': 'gc',
 'date': '',
 'result': '1',
 'gc': '',
 'icon': 'st7',
 'race_ref': 'race/tour-de-france/2021/stage-21-kom',
 'race_name': 'tour-de-france',
 'race_detail': 'Mountains classification',
 'race_rank': '',
 'distance': ''}

In [67]:
base_url = 'https://www.procyclingstats.com/'

endpoint = 'rider/tadej-pogacar/'

year = '2021'

rider = 'POGAČAR Tadej'

url = base_url+endpoint+year

In [68]:
response = requests.get(url).content

soup = BeautifulSoup(response)

In [77]:
stage_races = soup.find_all('tr', {'data-main': '0'})

one_day_races = soup.find_all('tr', {'data-main': '1'})

In [88]:
master_ls = []
for o in one_day_races:
    dict = {}
    o = o.find_all('td')
    dict['type'] = 'one_day'
    dict['day'] = o[0].text
    dict['result'] = o[1].text
    dict['race_ref'] = o[4].a['href']
    dict['race_name'] = o[4].a.text
    dict['race_rank'] = o[4].find_all('span')[-1].text
    dict['distance'] = o[5].text
    master_ls.append(dict)

In [89]:
pd.DataFrame(master_ls)

,day,result,race_ref,race_name,race_rank,distance
0,09.10,1,race/il-lombardia/2021/result,Il Lombardia (1.UWT),(1.UWT),239
1,06.10,4,race/milano-torino/2021/result,Milano - Torino (1.Pro),51k,190
2,05.10,3,race/tre-valli-varesine/2021/result,Tre Valli Varesine (1.Pro),86k,196.7
3,02.10,DNF,race/giro-dell-emilia/2021/result,Giro dell'Emilia (1.Pro),(1.Pro),195.3
4,26.09,37,race/world-championship/2021/result,World Championships - Road Race (WC),(WC),268.3
5,19.09,10,race/world-championship-itt/2021/result,World Championships - ITT (WC),(WC),43.3
6,12.09,5,race/uec-road-european-championships/2021/result,European Continental Championships - Road Race...,68k,179.2
7,09.09,12,race/uec-road-european-championships-itt/2021/...,European Continental Championships - ITT (CC),(CC),22.4
8,29.08,DNF,race/bretagne-classic/2021/result,Bretagne Classic - Ouest-France (1.UWT),(1.UWT),251
9,24.07,3,race/olympic-games/2021/result,Olympic Games Road Race (Olympics),(Olympics),234


In [113]:
stage_races[0].find('span', class_='icon')['class'][-1]

'st6'

In [90]:
master_ls = []
for o in stage_races:
    dict = {}
    o = o.find_all('td')
    dict['type'] = 'stage_race'
    dict['day'] = o[0].text
    dict['result'] = o[1].text
    dict['race_ref'] = o[4].a['href']
    dict['race_name'] = o[4].a.text
    dict['race_rank'] = o[4].find_all('span')[-1].text
    dict['distance'] = o[5].text
    master_ls.append(dict)

In [91]:
pd.DataFrame(master_ls)

,day,result,race_ref,race_name,race_rank,distance
0,,1,race/tour-de-france/2021/stage-21-youth,Youth classification,,
1,,1,race/tour-de-france/2021/stage-21-kom,Mountains classification,,
2,,8,race/tour-de-france/2021/stage-21-points,Points classification,,
3,,1,race/tour-de-france/2021/gc,General classification,,
4,18.07,72,race/tour-de-france/2021/stage-21,Stage 21 - Chatou › Paris Champs-Élysées,,108.4
...,...,...,...,...,...,...
59,25.02,2,race/uae-tour/2021/stage-5,Stage 5 - Fujairah Marine Club › Jebel Jais,,170
60,24.02,20,race/uae-tour/2021/stage-4,Stage 4 - Al Marjan Island › Al Marjan Island,,204
61,23.02,1,race/uae-tour/2021/stage-3,Stage 3 - Strata Manufactoring › Jebel Hafeet,,166
62,22.02,4,race/uae-tour/2021/stage-2,Stage 2 (ITT) - Al Hudayriat Island › Al Huday...,,13
